# Deep Agents Integration Demo (Claude API)

Демонстрация полной интеграции Deep Agents middleware в StatefulAgent с использованием Claude API.

## Компоненты:

1. **TodoListMiddleware** - управление task list через `write_todos` tool
2. **FilesystemMiddleware** - context eviction для больших результатов
3. **SubAgentMiddleware** - делегирование задач через `task` tool
4. **SummarizationMiddleware** - LLM-based сжатие контекста при превышении токенов
5. **File Tools** - edit_file, grep для работы с файлами

## Требования:

```bash
export ANTHROPIC_API_KEY="your-api-key-here"
```

In [4]:
# Setup imports
import sys
import os
import asyncio
from pathlib import Path

# Check for API key
if not os.getenv("ANTHROPIC_API_KEY"):
    print("⚠️  ANTHROPIC_API_KEY not found in environment!")
    print("   Set it with: export ANTHROPIC_API_KEY='your-key-here'")
else:
    print("✓ ANTHROPIC_API_KEY found")

# Add SDK to path
sys.path.insert(0, str(Path.cwd()))

from agentarea_agents_sdk.agents.stateful_agent import StatefulAgent
from agentarea_agents_sdk.tools.file_toolset import FileToolset
from agentarea_agents_sdk.middleware.summarization import count_messages_tokens

✓ ANTHROPIC_API_KEY found


## 1. Базовая инициализация StatefulAgent с middleware

По умолчанию StatefulAgent включает все Deep Agents middleware.

**Claude Models:**
- `claude-sonnet-4-5-20250929` - Latest Sonnet 4.5
- `claude-3-5-sonnet-20241022` - Sonnet 3.5
- `claude-3-opus-20240229` - Opus 3

In [3]:
# Создаём агента с всеми default middleware
agent = StatefulAgent(
    name="DeepAgent",
    instruction="You are a highly capable AI agent with planning and delegation abilities.",
    model_provider="anthropic",
    model_name="claude-sonnet-4-5",  # или claude-sonnet-4-5-20250929
    temperature=0.7,
    max_tokens=4096,
    max_iterations=15,
    enable_default_middleware=True,  # Включаем все middleware
    enable_todolist=True,
    enable_filesystem=True,
    enable_subagents=True,
    enable_summarization=True,
    max_tokens_before_summary=100_000,  # Claude поддерживает большой контекст
    messages_to_keep=6,
)

print(f"✓ Агент '{agent.name}' создан")
print(f"  Model: {agent.model_name}")
print(f"  Middleware: {[type(m).__name__ for m in agent.middlewares.middlewares]}")
print(f"  Tools: {[tool.name for tool in agent.tool_executor.registry.list_tools()]}")

✓ Агент 'DeepAgent' создан
  Model: claude-sonnet-4-5
  Middleware: ['TodoListMiddleware', 'FilesystemMiddleware', 'SubAgentMiddleware', 'SummarizationMiddleware']
  Tools: ['completion', 'write_todos', 'task']


## 2. TodoListMiddleware - планирование сложных задач

Агент использует `write_todos` tool для декомпозиции сложных задач.

In [4]:
# Задача, требующая планирования
task = """
Plan a simple pizza party for 10 people this Friday evening.

Requirements:
1. Create a shopping list (pizza ingredients, drinks, snacks)
2. Suggest 3 pizza varieties to make
3. List needed equipment and supplies
4. Create a simple timeline for preparation (2-3 hours before guests arrive)

Work step-by-step and use write_todos to track your progress.
"""

print("🎯 Задача:", task.strip())
print("\n" + "="*80 + "\n")

# Запускаем агента
async def run_planning_demo():
    result = ""
    async for chunk in agent.run_stream(task):
        print(chunk, end="", flush=True)
        result += chunk
    return result

result = await run_planning_demo()
print("\n" + "="*80)
print("\n✓ Задача выполнена")

🎯 Задача: Plan a simple pizza party for 10 people this Friday evening.

Requirements:
1. Create a shopping list (pizza ingredients, drinks, snacks)
2. Suggest 3 pizza varieties to make
3. List needed equipment and supplies
4. Create a simple timeline for preparation (2-3 hours before guests arrive)

Work step-by-step and use write_todos to track your progress.


**Thought:** I need to plan a pizza party for 10 people with multiple requirements: shopping list, pizza varieties, equipment, and a preparation timeline. This is a multi-step task with 4 distinct requirements, so I should use the write_todos tool to track my progress and ensure I complete each requirement systematically.

 The task has 4 clear requirements that need to be addressed. I have all the information needed to complete this task (number of people: 10, timing: Friday evening, preparation window: 2-3 hours). This is complex enough to warrant using write_todos for organization.

 a todo list to track each requirement, th

## 3. FilesystemMiddleware - работа с файлами

Демонстрация file tools: save_file, read_file, edit_file, grep.

In [5]:
import shutil

# Создаём тестовый проект
test_dir = Path("/tmp/demo_deep_agents")
if test_dir.exists():
    shutil.rmtree(test_dir)
test_dir.mkdir(parents=True, exist_ok=True)

# Агент с file tools
file_toolset = FileToolset(
    base_dir=test_dir,
    save_files=True,
    read_files=True,
    edit_files=True,
    grep_files=True,
)

agent_with_files = StatefulAgent(
    name="FileAgent",
    instruction="You are an agent that works with files.",
    model_provider="anthropic",
    model_name="claude-sonnet-4-5",
    temperature=0.5,
    max_tokens=4096,
    tools=[file_toolset],
    enable_default_middleware=True,
)

print(f"✓ Создан тестовый проект: {test_dir}")
print(f"✓ Агент с file tools: {[tool.name for tool in agent_with_files.tool_executor.registry.list_tools()]}")

✓ Создан тестовый проект: /tmp/demo_deep_agents
✓ Агент с file tools: ['completion', 'write_todos', 'file', 'task']


In [8]:
# Задача с файлами
file_task = """
1. Create a Python file 'config.py' with:
   - DATABASE_URL = "sqlite:///app.db"
   - API_KEY = "secret123"
   - DEBUG = True

2. Read the file back to verify

3. Replace 'secret123' with 'production_key_456'

4. Search for 'production' in all .py files
"""

print("🎯 Задача:", file_task.strip())
print("\n" + "="*80 + "\n")

async def run_file_demo():
    result = ""
    async for chunk in agent_with_files.run_stream(file_task):
        print(chunk, end="", flush=True)
        result += chunk
    return result

result = await run_file_demo()
print("\n" + "="*80)
print("\n✓ Файловые операции выполнены")

# Проверяем результат
config_path = test_dir / "config.py"
if config_path.exists():
    print("\n📄 Содержимое config.py:")
    print(config_path.read_text())

🎯 Задача: 1. Create a Python file 'config.py' with:
   - DATABASE_URL = "sqlite:///app.db"
   - API_KEY = "secret123"
   - DEBUG = True

2. Read the file back to verify

3. Replace 'secret123' with 'production_key_456'

4. Search for 'production' in all .py files


**Thought:** I need to complete a multi-step task involving creating a Python configuration file, verifying it, modifying it, and searching for a keyword. This task has 4 distinct steps that need to be completed sequentially.

 task is clearly defined with 4 separate steps. Since this is a multi-step task (more than 3 steps), I should create a plan using write_todos to track my progress and give the user visibility into what I'm doing.

:** I'll create a todo list to track these 4 steps, marking the first one as in_progress, then begin executing the plan.
[Tool write_todos: Updated todo list (4 tasks):
- [in_progress] Create config.py file with DATABASE_URL, API_KEY, and DEBUG variables
- [pending] Read config.py to verify c

## 4. SubAgentMiddleware - делегирование через task tool

Агент может делегировать подзадачи изолированным subagent'ам.

In [6]:
import os

# Создаем директорию
os.makedirs("/tmp/demo_deep_agents", exist_ok=True)

# Создаем example.py
with open("/tmp/demo_deep_agents/example.py", "w", encoding="utf-8") as f:
    f.write('''"""
Пример Python модуля для демонстрации.
"""


def calculate_sum(a: int, b: int) -> int:
    """Складывает два числа."""
    return a + b


def calculate_product(a: int, b: int) -> int:
    """Умножает два числа."""
    return a * b


class Calculator:
    """Простой калькулятор."""

    def __init__(self):
        self.history = []

    def add(self, a: int, b: int) -> int:
        result = a + b
        self.history.append(f"{a} + {b} = {result}")
        return result

    def multiply(self, a: int, b: int) -> int:
        result = a * b
        self.history.append(f"{a} * {b} = {result}")
        return result


# Константы
MAX_VALUE = 1000
MIN_VALUE = -1000
''')

# Создаем data.txt
with open("/tmp/demo_deep_agents/data.txt", "w", encoding="utf-8") as f:
    f.write('''Это тестовый файл с данными.

Список задач:
- Задача 1: Проанализировать example.py
- Задача 2: Написать отчет
- Задача 3: Сохранить результаты

Метрики:
Всего функций: 2
Всего классов: 1
Строк кода: ~30
''')

print("Тестовые файлы созданы в /tmp/demo_deep_agents/")

Тестовые файлы созданы в /tmp/demo_deep_agents/


In [9]:
# Создаём file toolset
from pathlib import Path
from agentarea_agents_sdk.tools.file_toolset import FileToolset

test_dir = Path("/tmp/demo_deep_agents")
# test_dir.mkdir(parents=True, exist_ok=True)

file_toolset = FileToolset(
    base_dir=test_dir,
    save_files=True,
    read_files=True,
    edit_files=True,
    grep_files=True,
)

# Определяем custom subagents
custom_subagents = [
    {
        "name": "analyzer",
        "description": "Specialized agent for code analysis and review",
        "system_prompt": "You are a code analysis expert. Analyze code thoroughly and provide detailed insights.",
    },
    {
        "name": "writer",
        "description": "Specialized agent for documentation and report writing",
        "system_prompt": "You are a technical writer. Create clear, well-structured documentation.",
    },
]

agent_with_subagents = StatefulAgent(
    name="OrchestratorAgent",
    instruction="You are an orchestrator that delegates tasks to specialized subagents.",
    model_provider="anthropic",
    model_name="claude-sonnet-4-5",
    temperature=0.7,
    max_tokens=4096,
    max_iterations=20,
    tools=[file_toolset],
    enable_default_middleware=True,
    enable_subagents=True,
    subagents=custom_subagents,
)

print("✓ Orchestrator Agent создан")
print(f"  Available subagents: general-purpose, analyzer, writer")
print(f"  Tools: {[tool.name for tool in agent_with_subagents.tool_executor.registry.list_tools()]}")

✓ Orchestrator Agent создан
  Available subagents: general-purpose, analyzer, writer
  Tools: ['completion', 'write_todos', 'file', 'task']


In [10]:
# Задача, требующая делегирования
delegation_task = """
Проанализируй файлы в директории /tmp/demo_deep_agents:

1. Найди и прочитай файл /tmp/demo_deep_agents/example.py
2. Определи основные компоненты (классы, функции, константы)
3. Прочитай файл /tmp/demo_deep_agents/data.txt для контекста
4. Напиши краткий отчет на русском языке о структуре модуля example.py
5. Сохрани отчет в файл /tmp/demo_deep_agents/analysis_report.txt

Работай пошагово, создай план и выполняй его.
"""

print("🎯 Задача с делегированием:", delegation_task.strip())
print("\n" + "="*80 + "\n")

async def run_delegation_demo():
    result = ""
    async for chunk in agent_with_subagents.run_stream(delegation_task):
        print(chunk, end="", flush=True)
        result += chunk
    return result

result = await run_delegation_demo()
print("\n" + "="*80)
print("\n✓ Делегирование завершено")

🎯 Задача с делегированием: Проанализируй файлы в директории /tmp/demo_deep_agents:

1. Найди и прочитай файл /tmp/demo_deep_agents/example.py
2. Определи основные компоненты (классы, функции, константы)
3. Прочитай файл /tmp/demo_deep_agents/data.txt для контекста
4. Напиши краткий отчет на русском языке о структуре модуля example.py
5. Сохрани отчет в файл /tmp/demo_deep_agents/analysis_report.txt

Работай пошагово, создай план и выполняй его.


**Thought:** Пользователь просит проанализировать файлы в директории /tmp/demo_deep_agents. Это многошаговая задача, которая включает:
тение файла example.py
ализ его компонентов
тение файла data.txt для контекста
ание отчета на русском языке
ение отчета в файл

 сложная задача с 5 четкими шагами, поэтому мне следует создать план с помощью write_todos.

 меня есть все необходимые инструменты для выполнения задачи: file для чтения/записи файлов, и возможность анализировать содержимое. Задача четко структурирована и требует пошагового выполнения

## 5. SummarizationMiddleware - управление контекстом

При превышении лимита токенов middleware автоматически сжимает историю.

**Claude Context Windows:**
- Claude 3.5 Sonnet: 200k tokens
- Claude Opus 3: 200k tokens

In [ ]:
# Агент с низким порогом для демонстрации
agent_with_summary = StatefulAgent(
    name="SummaryAgent",
    instruction="You are a helpful assistant.",
    model_provider="anthropic",
    model_name="claude-sonnet-4-5",
    temperature=0.7,
    max_tokens=2048,
    enable_default_middleware=True,
    enable_summarization=True,
    max_tokens_before_summary=2000,  # Низкий порог для демонстрации
    messages_to_keep=3,
)

print("✓ Агент с summarization создан")
print(f"  Token threshold: 2000")
print(f"  Messages to keep: 3")

In [ ]:
# Симулируем длинный разговор
conversations = [
    "Tell me about Python programming language in detail.",
    "What are the key features of Python? Explain each one.",
    "Explain Python's garbage collection mechanism in depth.",
    "How does Python's GIL work and why is it important?",
    "What are Python decorators and how do they work?",
    "Explain metaclasses in Python with examples.",
]

async def run_summary_demo():
    for i, question in enumerate(conversations, 1):
        print(f"\n{'='*80}")
        print(f"Question {i}: {question}")
        print(f"{'='*80}\n")
        
        async for chunk in agent_with_summary.run_stream(question):
            print(chunk, end="", flush=True)
        
        # Проверяем количество токенов
        messages = agent_with_summary.state.get("messages", [])
        token_count = count_messages_tokens(messages)
        print(f"\n\n📊 Current token count: {token_count}")
        print(f"📝 Messages in history: {len(messages)}")
        
        # Проверяем, был ли summary
        has_summary = any("summary" in str(msg.get("content", "")).lower() 
                         for msg in messages if msg.get("role") == "system")
        if has_summary:
            print("🔄 Summarization triggered!")

await run_summary_demo()
print("\n" + "="*80)
print("\n✓ Summarization demo завершено")

## 6. FilesystemMiddleware - Context Eviction

Большие результаты инструментов автоматически сохраняются в virtual FS.

In [ ]:
from agentarea_agents_sdk.middleware.filesystem import FilesystemMiddleware

# Тестируем context eviction напрямую
middleware = FilesystemMiddleware(eviction_threshold=100)
state = {"files": {}}

# Симулируем большой tool result
tool_call = {
    "id": "call_12345",
    "function": {"name": "search_codebase"},
}

large_result = "\n".join([f"Match {i}: Some code snippet here with lots of content..." for i in range(50)])

print(f"📊 Original result size: {len(large_result)} characters")
print(f"🎯 Eviction threshold: 100 characters")
print()

# Применяем middleware
result, state_update = await middleware.after_tool_call(tool_call, large_result, state)

print("📦 Result after middleware:")
print(result)
print()

if state_update and "files" in state_update:
    print("✓ Result evicted to virtual filesystem:")
    for path, content in state_update["files"].items():
        print(f"  {path}: {len(content)} characters")

## 7. Комплексная демонстрация всех компонентов

Финальный пример, объединяющий все middleware.

In [ ]:
# Создаём полноценного Deep Agent
deep_agent = StatefulAgent(
    name="FullDeepAgent",
    instruction="""You are a highly capable AI agent with:
    - Planning abilities (write_todos)
    - Task delegation (task tool for subagents)
    - File operations (save, read, edit, grep)
    - Context management (automatic summarization)
    
    Use all your capabilities effectively to complete complex tasks.
    """,
    model_provider="anthropic",
    model_name="claude-3-5-sonnet-20241022",
    temperature=0.7,
    max_tokens=4096,
    max_iterations=25,
    tools=[file_toolset],
    enable_default_middleware=True,
    enable_todolist=True,
    enable_filesystem=True,
    enable_subagents=True,
    enable_summarization=True,
    subagents=custom_subagents,
    max_tokens_before_summary=100_000,
    messages_to_keep=6,
)

print("✓ Full Deep Agent создан со всеми компонентами:")
print(f"  Model: {deep_agent.model_name}")
print(f"  Middleware: {[type(m).__name__ for m in deep_agent.middlewares.middlewares]}")
print(f"  Tools: {[tool.name for tool in deep_agent.tool_executor.registry.list_tools()]}")
print(f"  Subagents: general-purpose, analyzer, writer")

In [ ]:
# Комплексная задача
complex_task = """
I need you to complete a comprehensive project analysis:

1. Create a project plan (use write_todos to break down the task)

2. Delegate the code analysis to the 'analyzer' subagent:
   - Analyze the middleware architecture
   - Identify key patterns
   - Suggest improvements

3. Create a report file 'analysis_report.md':
   - Title and introduction
   - Analysis findings from subagent
   - Recommendations section
   - Conclusion

4. Search the report for the word 'middleware'

5. Provide a final summary
"""

print("🎯 Комплексная задача:", complex_task.strip())
print("\n" + "="*80 + "\n")

async def run_complex_demo():
    result = ""
    async for chunk in deep_agent.run_stream(complex_task):
        print(chunk, end="", flush=True)
        result += chunk
    return result

result = await run_complex_demo()
print("\n" + "="*80)
print("\n✅ Комплексная задача выполнена!")

# Проверяем созданный файл
report_path = test_dir / "analysis_report.md"
if report_path.exists():
    print("\n📄 Создан файл: analysis_report.md")
    print(f"   Размер: {len(report_path.read_text())} символов")

## 8. Статистика и анализ

Проверяем статистику работы агента.

In [ ]:
# Анализируем state агента
messages = deep_agent.state.get("messages", [])
iteration = deep_agent.state.get("iteration", 0)
files = deep_agent.state.get("files", {})

print("📊 Статистика Deep Agent:")
print(f"  Сообщений в истории: {len(messages)}")
print(f"  Итераций выполнено: {iteration}")
print(f"  Файлов в virtual FS: {len(files)}")
print(f"  Токенов в контексте: {count_messages_tokens(messages)}")

# Считаем вызовы инструментов
tool_calls = {}
for msg in messages:
    if msg.get("role") == "assistant" and "tool_calls" in msg:
        for tc in msg["tool_calls"]:
            tool_name = tc["function"]["name"]
            tool_calls[tool_name] = tool_calls.get(tool_name, 0) + 1

print("\n🔧 Использование инструментов:")
for tool_name, count in sorted(tool_calls.items(), key=lambda x: x[1], reverse=True):
    print(f"  {tool_name}: {count}x")

# Анализируем типы сообщений
message_types = {}
for msg in messages:
    role = msg.get("role", "unknown")
    message_types[role] = message_types.get(role, 0) + 1

print("\n💬 Распределение сообщений:")
for role, count in sorted(message_types.items(), key=lambda x: x[1], reverse=True):
    print(f"  {role}: {count}")

## 9. Сравнение с Deep Agents (LangChain)

Наша реализация vs оригинальная.

In [ ]:
comparison = {
    "Component": [
        "TodoListMiddleware",
        "FilesystemMiddleware",
        "SubAgentMiddleware",
        "SummarizationMiddleware",
        "File Tools",
        "Execute Tool",
        "Prompt Caching",
        "LLM Provider Support",
    ],
    "Deep Agents (LangChain)": [
        "✓ Full",
        "✓ Full",
        "✓ Full",
        "✓ 170k tokens",
        "✓ 7 tools",
        "✓ With sandbox",
        "✓ Anthropic only",
        "✓ Multi-provider",
    ],
    "Our Implementation": [
        "✓ Full",
        "✓ Context eviction",
        "✓ Full",
        "✓ 100k tokens",
        "✓ 6 tools",
        "✗ No sandbox",
        "✓ Via litellm",
        "✓ Any via litellm",
    ],
}

print("📋 Сравнение компонентов:\n")
print(f"{'Component':<30} {'Deep Agents':<25} {'Our Implementation':<25}")
print("="*80)

for i in range(len(comparison["Component"])):
    comp = comparison["Component"][i]
    da = comparison["Deep Agents (LangChain)"][i]
    ours = comparison["Our Implementation"][i]
    print(f"{comp:<30} {da:<25} {ours:<25}")

print("\n✅ Ключевые преимущества нашей реализации:")
print("  1. Независимость от LangChain/LangGraph")
print("  2. Поддержка любых LLM providers через litellm (Claude, GPT-4, Gemini, etc.)")
print("  3. Enable/disable флаги для гибкого контроля")
print("  4. Fallback механизм в summarization")
print("  5. Простая архитектура без лишних зависимостей")
print("  6. Prompt caching работает через litellm для всех providers")

print("\n⚠️ Компромиссы:")
print("  1. Нет execute tool (требует sandbox backend)")
print("  2. Нет специфичных для Anthropic оптимизаций")

## 10. Claude-specific Features

Демонстрация особенностей работы с Claude.

In [ ]:
# Claude excels at:
# 1. Long context understanding (200k tokens)
# 2. Complex reasoning and planning
# 3. Code analysis and generation
# 4. Following detailed instructions

claude_strengths_task = """
Demonstrate your capabilities:

1. Analyze this complex scenario:
   - A distributed system with 5 microservices
   - Each service has different failure modes
   - Need to design a resilient architecture

2. Create a detailed plan with:
   - Service communication patterns
   - Failure handling strategies
   - Monitoring and alerting
   - Rollback procedures

3. Break this down into actionable tasks using write_todos
"""

print("🎯 Claude Strengths Demo:", claude_strengths_task.strip())
print("\n" + "="*80 + "\n")

async def run_claude_demo():
    result = ""
    async for chunk in deep_agent.run_stream(claude_strengths_task):
        print(chunk, end="", flush=True)
        result += chunk
    return result

result = await run_claude_demo()
print("\n" + "="*80)
print("\n✓ Claude strengths demonstration complete")

## 11. Cleanup

In [ ]:
# Очищаем тестовые файлы
if test_dir.exists():
    shutil.rmtree(test_dir)
    print(f"✓ Удалён тестовый проект: {test_dir}")

print("\n✅ Demo завершено!")
print("\n🎉 Deep Agents integration with Claude API works perfectly!")